# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")


# Display sample data
city_data_df




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,39.94,76,92,1.01,US,1675710376
1,1,hermanus,-34.4187,19.2345,72.81,86,19,11.72,ZA,1675710716
2,2,wadi musa,30.3222,35.4793,37.26,89,88,24.32,JO,1675710781
3,3,vaini,-21.2000,-175.2000,78.73,73,100,2.35,TO,1675710710
4,4,lingao,19.9103,109.6859,69.66,92,3,11.36,CN,1675710784
...,...,...,...,...,...,...,...,...,...,...
347,347,inta,66.0317,60.1659,0.01,97,100,7.85,RU,1675710899
348,348,veriora,58.0079,27.3551,24.49,94,65,5.41,EE,1675711085
349,349,asau,46.4333,26.4000,24.28,92,81,6.02,RO,1675711086
350,350,lagoa,39.0500,-27.9833,55.87,71,96,28.19,PT,1675711087


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [101]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
cleaned_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 85) & (city_data_df["Max Temp"] >= 65) & (city_data_df["Wind Speed"] <= 5 ) & (city_data_df["Cloudiness"] <= 30)]


# Drop any rows with null values
cleaned_city_data_df = cleaned_city_data_df.dropna(how="any")

# Display sample data
cleaned_city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,tolaga bay,-38.3667,178.3000,65.57,81,6,3.09,NZ,1675710576
29,29,mount darwin,-16.7725,31.5838,67.53,92,2,2.37,ZW,1675710827
44,44,kasangulu,-4.5911,15.1708,75.00,82,18,4.21,CD,1675710834
45,45,cabo san lucas,22.8909,-109.9124,82.71,42,0,4.61,MX,1675710639
59,59,kapaa,22.0752,-159.3190,74.48,89,16,4.00,US,1675710775
61,61,esperance,-33.8667,121.9000,71.19,63,0,3.49,AU,1675710845
91,91,salalah,17.0151,54.0924,73.49,68,0,3.44,OM,1675710881
99,99,isangel,-19.5500,169.2667,79.57,83,20,4.56,VU,1675710885
179,179,georgetown,5.4112,100.3354,76.95,93,20,4.61,MY,1675710394
190,190,castro,-24.7911,-50.0119,79.65,49,14,4.29,BR,1675710650


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

/Users/fahmidabilla/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Lat,Lng,Humidity,Hotel Name
11,tolaga bay,NZ,-38.3667,178.3000,81,
29,mount darwin,ZW,-16.7725,31.5838,92,
44,kasangulu,CD,-4.5911,15.1708,82,
45,cabo san lucas,MX,22.8909,-109.9124,42,
59,kapaa,US,22.0752,-159.3190,89,
61,esperance,AU,-33.8667,121.9000,63,
91,salalah,OM,17.0151,54.0924,68,
99,isangel,VU,-19.5500,169.2667,83,
179,georgetown,MY,5.4112,100.3354,93,
190,castro,BR,-24.7911,-50.0119,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [94]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Get the city's name
    city = row["City"]

    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_response = name_address_response.json()
    name_address_response
    #pprint(name_address_response)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_response["features"][0]["properties"]["name"]
   
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


/Users/fahmidabilla/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


tolaga bay - nearest hotel: No hotel found
mount darwin - nearest hotel: No hotel found
kasangulu - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
kapaa - nearest hotel: Pono Kai Resort
esperance - nearest hotel: Hospitality
salalah - nearest hotel: Muscat International Hotel
isangel - nearest hotel: Tanna Lodge
georgetown - nearest hotel: Page 63 hostel
castro - nearest hotel: Hotel Central Palace
tchollire - nearest hotel: No hotel found
kariba - nearest hotel: Lodge
chiang klang - nearest hotel: Thungchang Hill Resort
port macquarie - nearest hotel: The Observatory
mapimi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
11,tolaga bay,NZ,-38.3667,178.3000,81,No hotel found
29,mount darwin,ZW,-16.7725,31.5838,92,No hotel found
44,kasangulu,CD,-4.5911,15.1708,82,No hotel found
45,cabo san lucas,MX,22.8909,-109.9124,42,Comfort Rooms
59,kapaa,US,22.0752,-159.3190,89,Pono Kai Resort
61,esperance,AU,-33.8667,121.9000,63,Hospitality
91,salalah,OM,17.0151,54.0924,68,Muscat International Hotel
99,isangel,VU,-19.5500,169.2667,83,Tanna Lodge
179,georgetown,MY,5.4112,100.3354,93,Page 63 hostel
190,castro,BR,-24.7911,-50.0119,49,Hotel Central Palace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [102]:
%%capture --no-display

# Configure the map plot

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity", 
    scale = 1,
    alpha = 0.5,
    color = "City",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2






:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)